In [0]:
from google.colab        import auth
from oauth2client.client import GoogleCredentials
import getpass

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

auth.authenticate_user()
creds = GoogleCredentials.get_application_default()

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import LabelBinarizer
from tensorflow              import keras
from keras.callbacks         import ModelCheckpoint
from keras.models            import Sequential
from keras.layers            import Dense, Flatten, Conv2D, Dropout
from os.path                 import join
from keras.optimizers        import Adam

import keras.backend     as K
import matplotlib.pyplot as plt
import tensorflow        as tf
import numpy             as np
import pandas            as pd
import seaborn           as sb
import pdb, os, random, keras


Using TensorFlow backend.


In [0]:
!rm /content/Weights*.hdf5

In [0]:
!pip3 list | grep tensorflow
!pip3 install tensorflow==1.13.1 # the binary is still in the 1.13.1 version

## Load Data


In [0]:
def xNormalize(value, max_value, min_value):
    return (value - min_value)/(max_value - min_value)

def yNormalize(value, max_value):
    if value < 0:
        return -1
    
    return value

def load_data2(directory):
    all_files = [join(directory, filename) for filename in os.listdir(directory)]
    
    random.shuffle(all_files)
    
    first_file = all_files.pop(0) 
    all_data   = pd.read_csv(first_file)
    
    for file in all_files:
        data = pd.read_csv(file)
        
        all_data = all_data.append(data)
        
    return all_data

def load_data(filepath):
    data = pd.read_csv(filepath)
    data = data.sample(frac=1)
    
    return data

def get_coordID_values(num_nodes):
    values = []
    
    for i in range(1, number_of_nodes + 1):
        values.append('CoordID_' + str(i))
        
    return values
    
def get_estimatePeerID_values(num_nodes):
    values = []
    
    for i in range(1, number_of_nodes + 1):
        values.append('EstimatePeerID_' + str(i))
        
    return values

def print_layer(layer, message, first_n=3, summarize=1024):
    return keras.layers.Lambda((
        lambda x: tf.Print(x, [x],
                           message=message,
                           first_n=first_n,
                           summarize=summarize)))(layer)
    

In [26]:
number_of_nodes          = 40
algorithm_variables      = 8
matrix_rows, matrix_cols = 1, (algorithm_variables - 4) + number_of_nodes*4 + 2 

filepath = 'sample_data/snapshots_' + str(number_of_nodes)  + '.csv'
data     = load_data(filepath)

train_set_size = int(len(data) * 0.8)

data_np = data.to_numpy()
delay   = data_np[:, (algorithm_variables + number_of_nodes):]

max_val_delay = np.amax(delay)
min_val_delay = np.amin(delay)

data_norm = pd.get_dummies(data, 
                           columns=['EstimateValue', 'Decision', 'PeerID', 'Phase', 'EstimatePeerID', 'CoordID'],
                           prefix=['EstimateValue', 'Decision', 'PeerID', 'Phase', 'EstimatePeerID', 'CoordID'])

current_estimatePeerID_values = list(filter(lambda x: 'EstimatePeerID' in x, data_norm.columns.values))
current_coordID_values        = list(filter(lambda x: 'CoordID'        in x, data_norm.columns.values))
current_phase_value           = list(filter(lambda x: 'Phase'          in x, data_norm.columns.values))

coordID_values        = get_coordID_values(number_of_nodes)
EstimatePeerID_values = get_estimatePeerID_values(number_of_nodes)
phase_values          = ['Phase_1', 'Phase_2']

phase_colums          = data_norm.T.reindex(phase_values).T.fillna(0)
coordID_columns       = data_norm.T.reindex(coordID_values).T.fillna(0)
estimatePeerID_colums = data_norm.T.reindex(EstimatePeerID_values).T.fillna(0)

data_norm.drop(current_estimatePeerID_values + current_coordID_values + current_phase_value, 
               inplace=True, 
               axis=1) 

data_norm = pd.concat([data_norm, coordID_columns, estimatePeerID_colums, phase_colums], axis=1)
data_np   = data_norm.to_numpy()

delays        = []
mut_variables = []

for row in data_np:
    splited_data    = np.split(row, [2 + number_of_nodes, 2 + (number_of_nodes*2)])
    aggregated_data = np.concatenate((splited_data[0], splited_data[2]), axis=None)
    
    mut_variables.append([aggregated_data])
    delays.append([splited_data[1]])

delays = [[[ yNormalize(element, max_val_delay) for element in list] for list in matrix] for matrix in delays]

delays        = np.asarray(delays)
mut_variables = np.asarray(mut_variables)

mut_variables_train = mut_variables[:train_set_size]
mut_variables_test  = mut_variables[train_set_size:]

delays_train = delays[:train_set_size]
delays_test  = delays[train_set_size:]

print('Done')

Done


In [0]:
C_mat = data.corr()
fig   = plt.figure(figsize=(50,50))

sb.heatmap(C_mat, vmax = 1., square = True)
#plt.show()
plt.savefig('foo2.png')

## Train, compile and fit model

In [0]:
batch_size = 16
sess       = tf.Session()  

K.set_session(sess)

# Activation function
activation = 'relu'

# Sequential model
model = Sequential()

# First layer
# The Input Layer :
model.add(Dense(256, 
                kernel_initializer='normal', 
                input_shape=(1, matrix_cols), 
                activation=activation, 
                name="input_layer"))

# The Hidden Layers :
model.add(Dense(128, kernel_initializer='normal',activation=activation))
model.add(Dense(128, kernel_initializer='normal',activation=activation))
model.add(Dense(128, kernel_initializer='normal',activation=activation))

# The Output Layer :
model.add(Dense(number_of_nodes, 
                kernel_initializer='normal', 
                activation='linear', 
                name="output_layer"))

# Compile the network :

# adam = Adam(lr=0.0001)
adam = 'adam'
model.compile(loss='mean_absolute_error', 
              optimizer=adam, 
              metrics=['mean_absolute_error'])
model.summary()

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint      = ModelCheckpoint(checkpoint_name, 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only = True, 
                                  mode ='auto')
callbacks_list = [checkpoint]

# Fit 
model.fit(mut_variables_train, 
          delays_train,
          batch_size=batch_size,
          epochs=100,
          validation_split=0.2,
          callbacks=callbacks_list)


In [0]:
model.summary()

In [0]:
print(data_norm.columns.values)

In [0]:
keras.backend.clear_session()

In [0]:
sess.run(tf.global_variables_initializer())

wights_file = 'Weights-070--0.02363.hdf5' # choose the best checkpoint 
adam        = 'adam'
#adam        = Adam(lr=0.0001)

model.load_weights(wights_file)  
model.compile(loss='mean_absolute_error', 
              optimizer=adam, 
              metrics=['mean_absolute_error'])


In [0]:
# Get input and output layer name
for n in sess.graph.as_graph_def().node:
    if 'input_layer' in n.name:
          print(n.name)
    if 'output_layer' in n.name:
          print(n.name)

In [0]:
predictions = model.predict(mut_variables_test)

for i in range(30):
    print(mut_variables_test[i][0][(4 + (number_of_nodes)):(4 + (number_of_nodes*2))])
    print(delays_test[i][0])
    print(predictions[i][0][:10])
    print("----------" + str(i))


In [0]:
# Use TF to save the graph model instead of Keras save model to load it in Golang
builder = tf.saved_model.builder.SavedModelBuilder("/content/drive/datasets/mut_model_40.5")  
# Tag the model, required for Go
builder.add_meta_graph_and_variables(sess, ["mut_tag"], clear_devices=True)  
builder.save()
# sess.close()